In [125]:
# imports
import pandas as pd
from sklearn.model_selection import KFold
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
import itertools
from matplotlib import cm as cm
import seaborn as sns

In [166]:
np.random.seed(42)
hsphere = pd.DataFrame([np.random.normal(size=100)*10 for i in range(5000)])
hsphere['label'] = ((hsphere[6]**2 + hsphere[20]**2 + hsphere[53]**2 + hsphere[23]**2 + hsphere[87]**2) <= 100) | (((hsphere[10] - 8)**2 + (hsphere[44] +3)**2 + (hsphere[53] - 5)**2) <= 25)

abalone = pd.read_csv('abalone.csv', header=None)

corral = pd.DataFrame()
np.random.seed(0)
corral['A0'] = pd.Series(np.random.randint(0, 2, 128))
np.random.seed(45)
corral['A1'] = pd.Series(np.random.randint(0, 2, 128))
np.random.seed(22)
corral['B0'] = pd.Series(np.random.randint(0, 2, 128))
np.random.seed(6)
corral['B1'] = pd.Series(np.random.randint(0, 2, 128))
np.random.seed(42)
corral['I'] = pd.Series(np.random.randint(0, 100, size=128))
clsss = pd.Series((corral['A0'] & corral['A1']) | (corral['B0'] & corral['B1']))
np.random.seed(0)
sample = clsss.sample(frac=.25).apply(lambda x: 1 if x == 0 else 0)
corral['C'] = clsss
corral['C'].loc[list(sample.index)] = sample
corral['label'] = clsss

usps = pd.read_csv('usps.csv')

arrhythmia = pd.read_csv('arrhythmia.csv', header=None)

gas = pd.read_csv('gas.csv')

segment = pd.read_csv('segment.csv')

spambase = pd.read_csv('spambase.csv')


(4597, 58)

In [2]:
# NEFS functions

def to_bin(arr):
    return int(''.join(arr.astype('int').astype('str')))


def get_hash_code(x, funcs):
    res = np.array(
        [math.floor((f['a'].dot(x) + f['b'])/f['w']) > 0 for f in funcs])
    return to_bin(res)


def generate_functions(dim, n_buckets=3, w=10, seed=42):
    As = np.random.standard_cauchy(1000)
    As = As[(As > -50) & (As < 50)]
    Bs = np.random.uniform(0, w + .1, 1000)

    np.random.seed(seed)

    functions = []
    for i in range(n_buckets):
        a = np.array([np.random.choice(As) for col in range(dim)])
        b = np.random.choice(Bs)
        functions.append({'a': a, 'b': b, 'w': w})

    return functions


def NEk(X, Y, lsh_index=None, k=4, n_buckets=3):
    S = X
    C = Y
    c_values = set(C.values)

    algo = 'kd_tree'
    knn = KNeighborsClassifier(n_neighbors=k, algorithm=algo, n_jobs=-1)
    knn.fit(S, C)
    neighbors = knn.kneighbors(S, n_neighbors=k, return_distance=False)

    entropy = 0
    for i in range(len(neighbors)):
        results = C.iloc[neighbors[i]].copy().append(
            C.iloc[[i]], ignore_index=False)
        freqs = results.value_counts(normalize=True)
        for clss in c_values:
            try:
                freq = freqs.loc[clss]
                entropy += freq * np.log(freq)
            except:
                pass
    return (- 1/len(S)) * entropy

#     if lsh_index == None:
#         funcs = generate_functions(dim=S.shape[1], n_buckets=n_buckets)
#         HASH = S.apply(lambda x: get_hash_code(x, funcs), axis=1)
#     else:
#         HASH = lsh_index.copy()

#     HASH.reindex(S.index)
    # start = datetime.now()
    # for index, row in S.iterrows():
    #     if index in visited:
    #         continue

    #     mask = HASH == HASH.loc[index]

    #     if S[mask].shape[0] >= k:
    #         rows = S[mask]
    #         labels = C[mask]
    #     else:
    #         rows = S
    #         labels = C

    #     knn.fit(rows, labels)

    #     neighbors = knn.kneighbors(
    #         [row], n_neighbors=k, return_distance=False)[0]
    #     results = labels.iloc[neighbors].copy().append(
    #         labels.loc[[index]], ignore_index=False)
    #     visited = np.append(visited, results.index)
    #     freqs = results.value_counts(normalize=True)

    #     for clss in c_values:
    #         try:
    #             freq = freqs.loc[clss]
    #             temp_sum += freq * np.log(freq)
    #         except:
    #             pass
    # delta = datetime.now() - start
    # print(f'lsh duration: {delta}')
    # return (- 1/len(rows)) * temp_sum


def NEFS(X, Y, k=4, s=2, n_buckets=3, single_indexed=False):
    if s > len(X.columns):
        raise Exception('"s" must be less or equal to number of columns in X')
    elif s == len(X.columns):
        return X

    F = X.copy()
    C = Y.copy()
    S = pd.DataFrame()

    if single_indexed:
        funcs = generate_functions(dim=S.shape[1], n_buckets=n_buckets)
        HASH = S.apply(lambda row: get_hash_code(row, funcs), axis=1)
    else:
        HASH = None

    while not (len(S.columns) == s):
        nes = {}
        for col in list(F):
            Scopy = S.copy()
            Scopy[col] = F[col]

            nes[col] = NEk(Scopy, C, k=k, n_buckets=n_buckets, lsh_index=HASH)

        min_col = min(nes, key=nes.get)
        S[min_col] = F[min_col]
        F.drop(columns=[min_col], inplace=True)
    return S

In [3]:
# RELIEFF functions

def RELIEFF(X, Y, k=5, iterations=50):
    S = X.copy()
    C = Y.copy()
    W = np.zeros(S.shape[1])
    
    algo = 'auto'
    knn = KNeighborsClassifier(algorithm=algo, n_jobs=-1)
    knn.fit(S, C)

    for it in range(iterations):
        sample = S.sample().iloc[0]

        dists, positions = knn.kneighbors([sample], n_neighbors=len(S), return_distance=True)
        positions = positions[0]

        hits = []
        misses = []
        for pos in positions:
            if C.loc[sample.name] == C.iloc[pos] and len(hits) < k:
                hits.append(S.iloc[pos])
            if C.loc[sample.name] != C.iloc[pos] and len(misses) < k:
                misses.append(S.iloc[pos])
            if len(misses) >= k and len(hits) >= k:
                break
    
        hit = np.mean(hits, axis=0)
        miss = np.mean(misses, axis=0)
        for i in range(len(W)):
            W[i] = W[i] - (sample[i] - hit[i])**2 + (sample[i] - miss[i])**2
    print(W)
    return [(w, W[w]) for w in range(len(W))]


In [25]:
# MI functions

def ENTROPY(Y):
    C = Y.copy()
    freqs = C.value_counts(normalize=True)
    entropy = 0
    for clss in set(C.values):
        freq = freqs.loc[clss]
        entropy += freq * np.log(freq)
    return -entropy


def COND_ENTROPY(Y, F):
    F_values = set(F.values)
    F_freqs = F.value_counts(normalize=True)
    C_values = set(Y.values)
    
    cond_entropy = 0
    
    for f in F_values:
        F_freq = F_freqs.loc[f]
        f_entropy = 0
        for clss in C_values:
            freqs = Y.loc[F[F == f].index].value_counts(normalize=True)
            try:
                freq = freqs.loc[clss]
                f_entropy += freq * np.log(freq)
            except:
                pass
        cond_entropy += F_freq * f_entropy
    
    return -cond_entropy


def MI(Y, F):
    return ENTROPY(Y) - COND_ENTROPY(Y, F)
        
    
def X_MIFS(X, Y, s):
    F = X.copy()
    mis = [(col, MI(Y, F[col])) for col in list(F.columns)]
    mis.sort(key=lambda tup: tup[1], reverse=True)
    cols = [col for col, mi in mis]
    return F[cols[:s]]


def mRMR_MIFS(X, Y, s):
    F = X.copy()
    S = pd.DataFrame()
    mis = [(col, MI(Y, F[col])) for col in list(F.columns)]
    mis.sort(key=lambda tup: tup[1], reverse=True)
    
    max_mi = mis[0][0]
    S[max_mi] = F[max_mi]
    F.drop(columns=[max_mi])
    
    S_cols_mis = {}
    S_cols_mis[max_mi] = {}
    while not (len(S.columns) == s):
        beta = 1/len(S.columns)
        scores = []
        for colF in list(F.columns):
                
            colF_mi = [mi for col, mi in mis if col == colF][0]
            colF_mis = []
            
            for colS in list(S.columns):
                if colF in S_cols_mis[colS]:
                    colF_mis.append(S_cols_mis[colS][colF])
                else:
                    temp_mi = MI(F[colF], S[colS])
                    colF_mis.append(temp_mi)
                    S_cols_mis[colS][colF] = temp_mi
                    
            scores.append((colF, colF_mi - beta * sum(colF_mis)))
        scores.sort(key=lambda tup: tup[1], reverse=True)
        max_mi = scores[0][0]
        S[max_mi] = F[max_mi]
        F.drop(columns=[max_mi])
        S_cols_mis[max_mi] = {}
        print(S_cols_mis)
        
    return S

In [23]:
# RRPC function
def RRPC(X, Y, s):
    S = pd.DataFrame()
    corrs = [(col, Y.corr(X[col], method='pearson')) for col in list(X.columns)]
    corrs.sort(key=lambda tup: tup[1], reverse=True)
    
    max_col = corrs.pop(0)[0]
    S[max_col] = X[max_col]
    
    while not (len(S.columns) == s):
        scores = []
        for col in list(X.columns):
            if col in list(S.columns): continue
            corr = Y.corr(X[col], method='pearson') - 1/len(S.columns) * sum([X[col].corr(S[colS], method='pearson') for colS in list(S.columns)])
            scores.append((col, corr))
        scores.sort(key=lambda tup: tup[1], reverse=True)
        max_col = scores.pop(0)[0]
        S[max_col] = X[max_col]
        
    return S

In [26]:
start = datetime.now()
print(f'start: {start.strftime("%H:%M:%S")}')

df = pd.read_csv('kc2.csv').dropna()
print(f'before removing duplicates: {len(df)}')
df.drop_duplicates(inplace=True)
print(f'after removing duplicates: {len(df)}')


# abalone
# df.iloc[:, 0] = df.iloc[:, 0].astype('category').cat.codes
# Y = pd.qcut(df.iloc[:, -1], q=4, labels=False)

X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
  
# for col in list(X.columns):
#       print(MI(Y, X[col]))
# RELIEFF(X, Y)

# main_2 = NEFS(X, Y, k=4, s=2, n_buckets=5)
main_2 = mRMR_MIFS(X, Y, 3)
print(main_2.head())

delta = datetime.now() - start
print(f'end: {datetime.now().strftime("%H:%M:%S")}')
print(f'duration: {delta}')

start: 02:20:31
before removing duplicates: 522
after removing duplicates: 375
t
{'t': {'loc': 4.226813192716841, 'v': 2.564375115590206, 'ev': 1.2129660832018994, 'iv': 2.2247082967134357, 'n': 4.952804508147159, 'v.1': 5.494366212290637, 'l': 3.2986878878708303, 'd': 5.210175224610827, 'i': 5.649874965834332, 'e': 5.6882381438475695, 'b': 3.7780786696911712, 't': 5.6919349288105545, 'lOCode': 4.110265182642865, 'lOComment': 1.8290054519363883, 'lOBlank': 2.6127179286491318, 'locCodeAndComment': 0.6750238336250695, 'uniq_Op': 3.0404460274747374, 'uniq_Opnd': 3.776140375088163, 'total_Op': 4.612232981908551, 'total_Opnd': 4.432603343728075, 'branchCount': 2.5632805081946275}, 'locCodeAndComment': {}}
{'t': {'loc': 4.226813192716841, 'v': 2.564375115590206, 'ev': 1.2129660832018994, 'iv': 2.2247082967134357, 'n': 4.952804508147159, 'v.1': 5.494366212290637, 'l': 3.2986878878708303, 'd': 5.210175224610827, 'i': 5.649874965834332, 'e': 5.6882381438475695, 'b': 3.7780786696911712, 't': 5.6

In [21]:
pd.Series(pd.qcut(range(25), 3, labels=['1quartil', '2quartil', '3quartil']).codes)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    2
18    2
19    2
20    2
21    2
22    2
23    2
24    2
dtype: int8